In [13]:
%pip -q install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
%pip -q install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.nn.utils.rnn import pack_sequence
import torch.nn.functional as F
import torch.nn.utils.rnn as rnn_utils
import pandas as pd
from torch.utils.data import DataLoader, Dataset

In [2]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
batch_size = 1
industry_size = 32
product_name_size = 20
product_category_size = 26

In [8]:
class LSTMComponent(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMComponent, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=input_dim, embedding_dim=hidden_dim)
        self.lstm = nn.LSTM(input_size=hidden_dim, hidden_size=hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, input):
        packed_input = pack_sequence(input, enforce_sorted=False)
        embedded_input = self.embedding(packed_input.data)
        packed_embedded = packed_input._replace(data=embedded_input)
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        return self.fc(hidden.squeeze(0))

class RecommenderModel(nn.Module):
    def __init__(self, industry_size, product_name_size, product_category_size, hidden_dim):
        super(RecommenderModel, self).__init__()

        self.industry_embedding = nn.Embedding(num_embeddings=industry_size, embedding_dim=32)
        self.lstm = LSTMComponent(product_name_size + product_category_size + 1, hidden_dim, 32)

        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, 10)

        self.dropout = nn.Dropout(p=0.2)

    def forward(self, industry, product_name, product_category, product_is_implemented):
        industry = self.industry_embedding(industry)
        product = torch.cat([product_name, product_category, product_is_implemented], dim=-1)
        product = self.lstm(product)

        print(industry.shape, product.shape)

        combined = torch.cat([industry, product], dim=1)

        x = F.relu(self.fc1(combined))
        x = self.dropout(x)
        output = self.fc2(x)

        return output

In [9]:
model = RecommenderModel(
    industry_size,
    product_name_size,
    product_category_size,
    hidden_dim=256
).to(device)

In [14]:
batch_size = 2
industry_1 = torch.randint(0, industry_size, (batch_size,), device=device)
product_1 = torch.randint(0, 2, (batch_size, 1), device=device)
product_2 = torch.randint(0, product_name_size, (batch_size, 1), device=device)
product_3 = torch.randint(0, product_category_size, (batch_size, 1), device=device)

In [15]:
result = model(industry_1, product_1, product_2, product_3)
result.shape

torch.Size([2, 32]) torch.Size([2, 32])


torch.Size([2, 10])

In [19]:
data = pd.read_csv('dataset.csv')

industry = data["Industry"].tolist()
product_name = data["ProductName"].tolist()
product_category = data["ProductCategory"].tolist()
product_is_implemented = data["ProductIsImplemented"].tolist()

class RecommenderDataset(Dataset):
    def __init__(self, input, output)

['Sports Equipment',
 'Sports Equipment',
 'Sports Equipment',
 'Sports Equipment',
 'Sports Equipment',
 'Sports Equipment',
 'Sports Equipment',
 'Sports Equipment',
 'Accessory Subscription',
 'Accessory Subscription',
 'Accessory Subscription',
 'Accessory Subscription',
 'Accessory Subscription',
 'Accessory Subscription',
 'Accessory Subscription',
 'Accessory Subscription',
 'Accessory Subscription',
 'Accessory Subscription',
 'Accessory Subscription',
 'Accessory Subscription',
 'Accessory Subscription',
 'Accessory Subscription',
 'Accessory Subscription',
 'Accessory Subscription',
 'Accessory Subscription',
 'Accessory Subscription',
 'Accessory Subscription',
 'Clothing Retailer',
 'Clothing Retailer',
 'Clothing Retailer',
 'Clothing Retailer',
 'Clothing Retailer',
 'Clothing Retailer',
 'Clothing Retailer',
 'Art Subscription Box',
 'Art Subscription Box',
 'Art Subscription Box',
 'Art Subscription Box',
 'Art Subscription Box',
 'Art Subscription Box',
 'Art Subscript